# JIF Tutorial

### JIF

JIF is a tool for organizing MCMC inference of posterior probability distributions of parametric astronomical image models.

https://github.com/mdschneider/JIF

*Follow the installation instructions in the README.*

### Spergel profile

JIF uses galsim to construct parametric models of galaxy images. The default parametric image model is the Spergel model, as opposed to the more well-known (and roughly equivalent) Sersic model. The Spergel profile is characterized by a shape parameter nu, a half light radius, and an overall flux level. In JIF we additionally specify the position of the galaxy's center (2 coordinates) and its ellipticity (2 components), for a total of seven parameters for each galaxy. The Spergel nu parameter plays a similar role to the Sersic n: nu = -0.6 is similar to an n = 4 Sersic profile (a DeVaucouleurs or "bulge" profile), while nu = 0.5 is equivalent to an n = 1 Sersic profile (an Exponential or "disk" profile). In galsim, the allowed range of values for the nu parameter is -0.85 <= nu <= 4.

This profile was chosen because of the way galsim computes things: "Sersic profile calculations take advantage of Hankel transform tables that are precomputed for a given value of n when the Sersic profile is initialized. Making additional objects with the same n can therefore be many times faster than making objects with different values of n that have not been used before." ( https://galsim-developers.github.io/GalSim/_build/html/gal.html#sersic-profile ) "Due to its analytic Fourier transform and depending on the indices n and ν, the Spergel profile can be considerably faster to draw than the roughly equivalent Sersic profile. ...for example, if drawing Sersic profiles with n continuously varying near 4.0 and Spergel profiles with ν continuously varying near -0.6, then the Spergel profiles are about 50x faster to draw." ( https://galsim-developers.github.io/GalSim/_build/html/gal.html#spergel-profile )

*Generate an example image:*

`cd jiffy`

`python galsim_galaxy.py`

*This will make the file `test_image_data.h5` in `data/TestData`. It's a Spergel profile with a tiny amount of random noise added.*

Python code to take an existing image (stored as a numpy array), along with a corresponding variance and mask plane, and convert it to the input format JIF expects:

In [ ]:
'''
Take a preexisting image and prepare the necessary inputs to JIF.
'''
def ingest_image(data_dir, name, pix_scale=0.2, band='i'):
    image = np.load(f'{data_dir}/image.npy')
    var = np.load(f'{data_dir}/var.npy')
    mask = np.load(f'{data_dir}/mask.npy')

    fname = f'{data_dir}/jiffy_real_image_{name}'
    ftpnt = footprints.Footprints(fname+'.h5')
    dummy_background = 0.0
    ftpnt.save_images([image], [var], [mask], [dummy_background],
        segment_index=0, telescope='LSST', filter_name=band)
    ftpnt.save_tel_metadata(pixel_scale_arcsec=pix_scale)

In JIF, the seven parameters are restricted to lie within fixed finite ranges indicated here: https://github.com/mdschneider/JIF/blob/master/jiffy/galsim_galaxy.py#L38. The parameters are also subject to additional constraints indicated here: https://github.com/mdschneider/JIF/blob/master/jiffy/galsim_galaxy.py#L47. The user can modify these ranges and constraints to suit their problem, but it's good to leave them as broad as possible so you're not ignoring portions of parameter space.

### emcee

JIF currently uses emcee \[ https://emcee.readthedocs.io/en/stable/ \] for MCMC sampling. emcee is a Python implementation of Goodman & Weare's affine-invariant MCMC ensemble sampler. \[ https://msp.org/camcos/2010/5-1/p04.xhtml \] It needs relatively little tuning in order to run decently fast on this task.

It starts by initializing a set of parallel "walkers", which each store a tuple of values representing the parameters of the galaxy image model. JIF lets you set the number of walkers - for this problem, 16 is a good number to start with. The walkers are initialized to some specific values (by default, JIF randomly initializes them within a small ball around some chosen point, so the user only needs to choose a single initial point for the entire set of walkers). Then the walkers are all updated in parallel for some number of steps, following Goodman & Weare's algorithm. `jiffy_roaster` takes care of initializing and running the chain.

*Run roaster on the example image:*

`jiffy_roaster --config_file ../config/jiffy.yaml --verbose`

*This will create an hdf5 results file `../output/TestData/jiffy_roaster_out_seg0.h5` containing the full MCMC chain.*

At every step of each MCMC chain, galsim draws a Spergel profile for each walker based on that walker's parameter values. JIF sets up the galsim light profile here: https://github.com/mdschneider/JIF/blob/master/jiffy/galsim_galaxy.py#L210

More advanced users can set up their own wrapper script to capture input arguments and send those to roaster after doing other kinds of custom processing. Here's an example of how to capture some arguments in a Python script and route those to roaster:

In [ ]:
# You'd run this in the main method of a python script.

import argparse

parser = argparse.ArgumentParser()
parser.add_argument('--config_file', type=str,
                    default="../config/jiffy_multirun.yaml",
                    help="Name of a configuration file listing inputs.")
parser.add_argument("--unparallelize", action='store_true',
                    help="Disable parallelizing during sampling")
parser.add_argument('--cluster_walkers', action='store_true',
                    help='Throw away outlier walkers.')
parser.add_argument('--cluster_walkers_thresh', type=float, default=4,
                    help='Threshold multiplier for throwing away walkers.')
parser.add_argument('--initialize_from_image', action='store_true',
                    help='Use image characteristics to set initial parameter values.\
                     So far only tested on centered, isolated galaxies.')
parser.add_argument("--footprint_number", type=int, default=0,
                        help="The footprint number to load from input")
parser.add_argument('--verbose', action='store_true',
                    help="Enable verbose messaging")

args = parser.parse_args()

In [ ]:
# Then the args can be passed to a roaster class instance that you set up in Python.

from jiffy import roaster

rstr = roaster.init_roaster(args)

In [ ]:
# Initializing the roaster above with custom initial parameter values:

rstr_params = {'nu': 0.0, 'hlr': 0.2, 'e1': 0.0, 'e2': 0.0, 'flux': 3.0, 'dx': 0.0, 'dy': 0.0}
for paramname in rstr_params:
    rstr.set_param_by_name(paramname, rstr_params[paramname])

In [ ]:
# Run the MCMC sampling
roaster.do_sampling(args, rstr)

### Number of steps

#### Convergence

The number of steps in the chain needs to be large enough to achieve "convergence" - this can be indicated by a Gelman-Rubin statistic close to 1.0, an integrated autocorrelation time in each parameter above 50, posterior means and variances that remain stable with increasing numbers of steps, etc. JIF can report the Gelman-Rubin statistic by running `jiffy_roaster_inspector`. `jiffy_roaster` also stores the full MCMC chains, allowing the user to look at them any way they want. For fitting a single galaxy with 16 walkers, posterior parameters can be very robustly inferred after 6000 total MCMC steps.

*Inspect the MCMC chain:*

`jiffy_roaster_inspector ../output/TestData/jiffy_roaster_out_seg0.h5 ../config/jiffy.yaml --verbose`

*This will print summary statistics and make some plots in `output/TestData`.*

In [ ]:
# args for roaster_inspector

parser = argparse.ArgumentParser()
parser.add_argument("--infile", type=str,
                        default='../output/TestData/jiffy_roaster_out_multirun_gal0_seg0.h5',
                        help="input HDF5 file with samples from Roaster")
parser.add_argument('--roaster_config', type=str,
                        default="../config/jiffy_multirun.yaml",
                        help="Name of a configuration file listing inputs.")
parser.add_argument("--roaster_config", type=str,
                    default='../config/jiffy_multirun.yaml',
                    help="Name of Roaster config file")
parser.add_argument("--keeplast", type=int, default=0,
                    help="Keep last N samples.")

args = parser.parse_args()

In [ ]:
# roaster_inspector converts the raw roaster outputs into a more usable format.

from jiffy import roaster_inspector

inspector = roaster_inspector.RoasterInspector(args)

In [ ]:
'''
Gather the MCMC sampled values for each parameter.

Input
args: Parsed command-line arguments

Returns
run_record: A dict containing, for every galaxy model parameter name,
    an array consisting of the lists of parameter values sampled at every MCMC
    step by every walker.
run_logprob: An array consisting of the lists of logprob values estimated at
    every MCMC step for every walker.
Only the last "keeplast" records of every run are kept in each returned array.
'''
def inspect(args):
    inspector = roaster_inspector.RoasterInspector(args)
    if args.verbose:
        inspector.summary()
        inspector.report()
    # inspector.save_param_cov()
    run_record = dict()
    for i, param in enumerate(inspector.paramnames):
        run_record[param] = inspector.data[-inspector.args.keeplast:, :, i]
    run_logprob = inspector.logprob[-inspector.args.keeplast:, :]
    return run_record, run_logprob

#### Burn in

Posterior inference can be made more stable and accurate by ignoring some number of initial steps. This number of "burn in" steps must be tuned for one's specific problem, but something like 20% of the total chain length is a good place to start. For fitting a single galaxy with 16 walkers for 6000 total MCMC steps, it's safe to ignore the first 1000 steps as "burn in" and just use the last 5000 for posterior inference.

### Generative model

Like any MCMC sampler, the user must specify a means for computing the prior probability of any given parameter values, along with a means for computing the likelihood of observing a given data vector given those parameter values. More specifically, the user must provide the natural logs of the prior and likelihood.

#### Prior

The example config tells JIF to use an "`Empty`" prior, which simply assumes a uniform distribution over all possible parameter values. A somewhat more complicated, but more realistic, prior for isolated galaxies is the `IsolatedFootprintPrior`. \[ https://github.com/mdschneider/JIF/blob/master/jiffy/priors.py#L7 \] The specific means and (co)variances stored in that prior were computed using DC2 data.

#### Likelihood

The likelihood, assuming independent Gaussian noise in each pixel with variance specified in the supplied variance plane, is computed here: https://github.com/mdschneider/JIF/blob/master/jiffy/roaster.py#L278